<a href="https://colab.research.google.com/github/eplg/Reveal_tech_case/blob/main/Technical_Case_Reveal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Définition de variables

In [48]:
THRESHOLD = 1.0
columns_names = ["Number", "Company", "Website","No data","Telephone","Adresse","Zip","City","Country"]
columns_to_compute = ["Company","Adresse","Zip","City","Country"]
columns_to_show = ["Adresse","Zip","City", "Country"]
columns_to_drop = ["Number","No data"]

Initialisation et préparation des datasets

In [45]:
dataset_A = pd.read_csv("/content/sample_data/dataset_A.csv", header=None, names=columns_names)
dataset_B = pd.read_csv("/content/sample_data/dataset_B.csv", header=None, names=columns_names)

dataset_A = dataset_A.drop(columns=columns_to_drop)
dataset_B = dataset_B.drop(columns=columns_to_drop)

dataset_A = dataset_A.fillna('')
dataset_B = dataset_B.fillna('')


**Solution 1**

In [52]:
merged_datasets = pd.merge(dataset_A, dataset_B, on=columns_to_compute, how='inner')
merged_datasets.fillna('')
merged_datasets = merged_datasets.drop_duplicates()
merged_datasets.to_csv('matchs_companies_solution1.csv', index=False)
merged_datasets.head()

,Company,Website_x,Telephone_x,Adresse,Zip,City,Country,combined_attributes_x,Website_y,Telephone_y,combined_attributes_y
0,LA CHOPE DAGUERRE,,01 43 22 76 59,,75014.0,,France,LA CHOPE DAGUERRE 75014.0 France,,01 43 22 76 59,LA CHOPE DAGUERRE 75014.0 France
1,TOT OU TARD,,,,34000.0,,France,TOT OU TARD 34000.0 France,,,TOT OU TARD 34000.0 France
2,Bagelstein Pyramides,,01 42 61 06 32,,75001.0,,,Bagelstein Pyramides 75001.0,,01 42 61 06 32,Bagelstein Pyramides 75001.0
3,Les Amis de Messina (PARENT),,01 42 61 13 73,,75002.0,,,Les Amis de Messina (PARENT) 75002.0,,01 42 61 13 73,Les Amis de Messina (PARENT) 75002.0
4,O Charolais,,01 34 64 02 17,,95310.0,,France,O Charolais 95310.0 France,,01 34 64 02 17,O Charolais 95310.0 France


**Solution 2**

In [49]:
## Fonction qui calcule la similarité entre deux datasets
def compute_similarity(dataset_A, dataset_B):
    vectorizer = TfidfVectorizer()

    # Concaténation des attributs des colonnes sélectionnées dans une nouvelle colonne
    dataset_A['combined_attributes'] = dataset_A[columns_to_compute].astype(str).agg(' '.join, axis=1)
    dataset_B['combined_attributes'] = dataset_B[columns_to_compute].astype(str).agg(' '.join, axis=1)

    # Calcul de la similarité des deux datasets
    tfidf_matrix1 = vectorizer.fit_transform(dataset_A['combined_attributes'])
    tfidf_matrix2 = vectorizer.transform(dataset_B['combined_attributes'])
    similarity_matrix = cosine_similarity(tfidf_matrix1, tfidf_matrix2)

    # Conversion de la matrice en dataframe pour l'analyser par la suite
    similarity_df = pd.DataFrame(similarity_matrix, index=dataset_A['Company'], columns=dataset_B['Company'])

    return similarity_df

## Fonction qui trouve les matches entre deux datasets
def find_matches(dataset_A, dataset_B):
    similarity_df = compute_similarity(dataset_A, dataset_B)

    dataset_A = dataset_A.set_index('Company')
    dataset_B = dataset_B.set_index('Company')

    # Recherche des entreprises qui matchent avec un score supérieur au seuil défini
    matches = []
    for company_A, row in similarity_df.iterrows():
        for company_B, score in row.items():
            if score >= THRESHOLD:
                match = {
                    'company_A': company_A,
                    'company_B': company_B,
                    'score': score
                }

                #Ajout d'attributs supplémentaires pour chaque dataset afin de valider les matchs
                match.update({f'{col}_A': val for col, val in dataset_A.loc[company_A, columns_to_show].to_dict().items()})
                match.update({f'{col}_B': val for col, val in dataset_B.loc[company_B, columns_to_show].to_dict().items()})
                matches.append(match)

    # Conversion en dataframe
    matches_companies = pd.DataFrame(matches)

    return matches_companies



In [53]:
matches_companies = find_matches(dataset_A, dataset_B)
matches_companies.to_csv('matches_companies_solution_2.csv', index=False)
matches_companies.head()

,company_A,company_B,score,Adresse_A,Zip_A,City_A,Country_A,Adresse_B,Zip_B,City_B,Country_B
0,LA CHOPE DAGUERRE,LA CHOPE DAGUERRE,1.0,,75014.0,,France,,75014.0,,France
1,TOT OU TARD,TOT OU TARD,1.0,,34000.0,,France,,34000.0,,France
2,La Dolce VIta,LA dolce Vita,1.0,,80000.0,,France,,80000,,France
3,Bagelstein Pyramides,Bagelstein Pyramides,1.0,,75001.0,,,,75001.0,,
4,Les Amis de Messina (PARENT),Les Amis de Messina (PARENT),1.0,,75002.0,,,,75002.0,,
